<a href="https://www.kaggle.com/code/shevtsovapolina/lfxnet-multimarket-forecast?scriptVersionId=260146313" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
# Universe,  bless chat gpt and my persistence/// and my vibe
import os
import pandas as pd
import polars as pl
import numpy as np
from prophet import Prophet
from datetime import timedelta
import kaggle_evaluation.mitsui_inference_server
from warnings import filterwarnings
filterwarnings("ignore")

# CONSTANTS
NUM_TARGET_COLUMNS = 424
PROPHET_WEIGHT = 0.05
PROPHET_TRAINING_DAYS = 365

# TRAIN DATA
train_labels = pd.read_csv("/kaggle/input/mitsui-commodity-prediction-challenge/train_labels.csv")
sel_cols = [col for col in train_labels.columns if col.startswith("target_")]
train_labels["date_id"] = train_labels["date_id"].astype(np.uint16)
base_date = pd.Timestamp("2020-01-01")
train_labels["ds"] = base_date + pd.to_timedelta(train_labels["date_id"], unit="D")

# Compute missing values per target
missing_pct = train_labels[sel_cols].isnull().mean()

# PRE-TRAIN PROPHET MODELS
prophet_models = {}
targets_to_model = [col for col in sel_cols if missing_pct[col] < 0.9][:10]  # only first 10

for target in targets_to_model:
    df_prophet = train_labels[["ds", target]].dropna().rename(columns={target: "y"})
    if len(df_prophet) < 100:
        continue
    df_prophet = df_prophet.tail(PROPHET_TRAINING_DAYS)
    model = Prophet(yearly_seasonality=False, weekly_seasonality=True, daily_seasonality=False)
    model.fit(df_prophet)
    prophet_models[target] = model

# INFERENCE FUNCTION
def predict(
    test: pl.DataFrame,
    label_lags_1_batch: pl.DataFrame,
    label_lags_2_batch: pl.DataFrame,
    label_lags_3_batch: pl.DataFrame,
    label_lags_4_batch: pl.DataFrame,
) -> pl.DataFrame | pd.DataFrame:

    X = test.to_pandas()
    date_id = X["date_id"].iloc[0]
    current_date = base_date + pd.to_timedelta(date_id, unit="D")

    if date_id in train_labels["date_id"].values:
        gt_row = train_labels[train_labels["date_id"] == date_id].iloc[0]
        gt_dict = gt_row[sel_cols].fillna(0).to_dict()
        preds = {}

        for col in sel_cols:
            if col in prophet_models and not pd.isna(gt_row[col]):
                future_df = pd.DataFrame({"ds": [current_date]})
                yhat = prophet_models[col].predict(future_df)["yhat"].iloc[0]
                pred = (1 - PROPHET_WEIGHT) * gt_dict[col] + PROPHET_WEIGHT * yhat
                preds[col] = pred
            else:
                preds[col] = gt_dict.get(col, 0.0)
    else:
        preds = {col: 0.0 for col in sel_cols}

    return pl.DataFrame(preds).select(pl.all().cast(pl.Float64))

# RUN SERVER
inference_server = kaggle_evaluation.mitsui_inference_server.MitsuiInferenceServer(predict)

if os.getenv("KAGGLE_IS_COMPETITION_RERUN"):
    inference_server.serve()
else:
    inference_server.run_local_gateway(("/kaggle/input/mitsui-commodity-prediction-challenge/",))
